Codul de antrenare pentru resnet task 1 si task 2 (kaggle)
Predictiile se fac pe local, de aceea salvez state dict-ul modelului, nu modelul in sine 

In [19]:
from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnet18
import torchvision
import numpy as np

import cv2 as cv
import torch

import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
import os

DIR_TRAIN = "/kaggle/input/task-1-2-dataset"
DIR_CHARACTERS = "characters"
POSITIVES = "positives"
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()
transform_flip_h = torchvision.transforms.RandomHorizontalFlip(p=0.5)

In [20]:
mapping = {
    "barney": 0,
    "betty": 1,
    "fred": 2,
    "wilma": 3,
    "other": 4
}

def getMapNameToLabel(image_name):
    if image_name.split("_")[0] in mapping:
        return mapping[image_name.split("_")[0]]
    else:
        return 4

In [33]:
import random

class FFDataset(Dataset):
    def __init__(self, character, transform=transform_flip_h):
        self.characters = ["fred", "barney", "wilma", "betty"]
        self.character = character
        self.transform = transform
        self.images = []
        self.labels = []
        if character is not None: #task 2
            print("task2")
            self.load_images(character=character, positives=True, task_1=False)
            self.ex_positive= len(self.images)
            print("gata exemplele pozitive: ", self.ex_positive)
            self.ex_negative= 2 * self.ex_positive
            self.load_images(character=character, positives=False, task_1=False)
            print("gata exemplele negative: ", self.ex_negative)
            print("all: ", len(self.images))
        else: # task 1
            self.load_images(task_1=True, positives=True)
            self.ex_positive= len(self.images)
            self.load_images(task_1=True, character=None)
            self.ex_negatives = len(self.images) - self.ex_positive
            

    def load_images(self, task_1=False, positives=False, character=None):
        path_dirs = []
        if task_1 == False: # task 2
            print("task 2")
            if positives and character is not None:
                print("task2_ character")
                path_dirs = [os.path.join(DIR_TRAIN, DIR_CHARACTERS, character)]
            else:
                other_characters = [ch for ch in self.characters if ch != character]
                path_dirs = [os.path.join(DIR_TRAIN, DIR_CHARACTERS, ch) for ch in other_characters]
                path_dirs = path_dirs + [os.path.join(DIR_TRAIN, DIR_CHARACTERS, "unknown"),
                             os.path.join(DIR_TRAIN, "negatives")]
                examples = 0
                print(examples)
        else: # task 1
            if positives:
                path_dirs = [os.path.join(DIR_TRAIN, "positives")]
            else:
                path_dirs = [os.path.join(DIR_TRAIN, "negatives")]
                examples = 0


        for path_dir in path_dirs:
            print("incarcam imag din : ", path_dir)
            files = os.listdir(path_dir)
            if positives==False and task_1==False:
                files = random.choices(files, k=len(files) // 3)
            for image_name in files:
                image = cv.imread(os.path.join(path_dir, image_name))
                img_test= Image.fromarray(image)
                t_img = Variable(normalize(to_tensor(scaler(img_test))))
                self.images.append(t_img)
                
                label = 0
                if positives:
                    label = 1
                else:
                    examples += 1
                
                self.labels.append(label)
                
                if (positives==False and task_1==False) and self.ex_negative <= examples:
                    print("gata exemplele negative")
                    return
                    

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label



In [56]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_EPOCHS = 50 # for task1 30, for task2 50

my_model = resnet18(num_classes=2).to(DEVICE)
cross_entropy_loss = torch.nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.SGD(my_model.parameters(),momentum=0.9, lr=1e-3, weight_decay=0.001)

In [57]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data.sampler import SubsetRandomSampler

dataset = FFDataset('wilma') # task 2, pt task 1 nu se pune nimic <-> dataset = FFDataset(None)
train_data = DataLoader(dataset, batch_size=32, shuffle=True)

batch_size = 32
validation_split = .20
shuffle_dataset = True
random_seed= 42
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                 sampler=valid_sampler)


task2
task 2
task2_ character
incarcam imag din :  /kaggle/input/task-1-2-dataset/characters/wilma
gata exemplele pozitive:  1605
task 2
0
incarcam imag din :  /kaggle/input/task-1-2-dataset/characters/fred
incarcam imag din :  /kaggle/input/task-1-2-dataset/characters/barney
incarcam imag din :  /kaggle/input/task-1-2-dataset/characters/betty
incarcam imag din :  /kaggle/input/task-1-2-dataset/characters/unknown
incarcam imag din :  /kaggle/input/task-1-2-dataset/negatives
gata exemplele negative
gata exemplele negative:  3210
all:  4815


In [58]:
#train
my_model.train(True)
loss_mean=0
losses=0
for epoch in range(NUM_EPOCHS):
    print(f"--Epoch {epoch+1}--")
    losses = 0
    loss_mean=0
    for batch, (images, labels) in enumerate(train_loader):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        out = my_model(images)
        loss = cross_entropy_loss(out, labels)
        loss_mean += loss
        losses += 1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 50 == 0:
            print(f"Batch index {batch}; learning rate {optimizer.param_groups[0]['lr']}; loss: {loss.item():>8f}")
            
    print(f"mean loss: {loss_mean/losses}")

--Epoch 1--
Batch index 0; learning rate 0.001; loss: 0.735953
Batch index 50; learning rate 0.001; loss: 0.169489
Batch index 100; learning rate 0.001; loss: 0.094984
mean loss: 0.28507646918296814
--Epoch 2--
Batch index 0; learning rate 0.001; loss: 0.058144
Batch index 50; learning rate 0.001; loss: 0.251812
Batch index 100; learning rate 0.001; loss: 0.034581
mean loss: 0.0942503958940506
--Epoch 3--
Batch index 0; learning rate 0.001; loss: 0.188081
Batch index 50; learning rate 0.001; loss: 0.009487
Batch index 100; learning rate 0.001; loss: 0.012361
mean loss: 0.08667925000190735
--Epoch 4--
Batch index 0; learning rate 0.001; loss: 0.079215
Batch index 50; learning rate 0.001; loss: 0.041887
Batch index 100; learning rate 0.001; loss: 0.105426
mean loss: 0.0726318508386612
--Epoch 5--
Batch index 0; learning rate 0.001; loss: 0.108980
Batch index 50; learning rate 0.001; loss: 0.025751
Batch index 100; learning rate 0.001; loss: 0.003034
mean loss: 0.057843852788209915
--Epoc

In [59]:
#validare
correct = 0
test_loss = 0
size = len(validation_loader.dataset) * validation_split
val_labels = []
predicted_val = []
my_model.to(DEVICE)
my_model.eval()
with torch.no_grad():
    for image_batch, labels_batch in validation_loader:
        image_batch = image_batch.to(DEVICE)
        labels_batch = labels_batch.to(DEVICE)
        pred = my_model(image_batch) 
        test_loss += cross_entropy_loss(pred, labels_batch).item()
        correct += (pred.argmax(1) == labels_batch).type(torch.float).sum().item()
        predicted_val.extend(pred.argmax(1))
        val_labels.extend(labels_batch)

correct /= size
test_loss /= size
print(f"Accuracy: {(100*correct):>0.1f}%, Loss: {test_loss:>8f} \n")

Accuracy: 99.3%, Loss: 0.000873 



In [62]:
wd = "/kaggle/working/"
torch.save(my_model.state_dict(), wd + "model_dict_3_2_1")